In [1]:
# Just to know last time this was run:
import time
print(time.ctime())

Thu Nov  3 14:31:27 2022


In [2]:
%load_ext scalene

Scalene extension successfully loaded. Note: Scalene currently only
supports CPU+GPU profiling inside Jupyter notebooks. For full Scalene
profiling, use the command line version.


# H Optimizing code writing

This is part of the Python lecture given by Christophe Morisset at IA-UNAM. More informations at: https://github.com/Morisset/Python-lectures-Notebooks

In [3]:
import numpy as np
import os
from urllib.request import urlopen
from scipy.integrate import simps

### Profiling the code: CPU usage

In [4]:
%%writefile test_1_prof.py

import numpy as np
import os
from urllib.request import urlopen
from scipy.integrate import simps

class Stel_Spectrum(object):
    """
    This object downloads a file from http://astro.uni-tuebingen.de/~rauch/TMAF/NLTE/He+C+N+O/
    and is able to make some plots.    
    """
    
    spec_count = 0 # This attibute is at the level of the class, not of the object.
    def __init__(self, filename=None, T=None, logg=None, verbose=False):
        """
        Initialisation of the Stel_Spectrum object.
        Parameter:
            - filename
            - T: temperature in K, e.g. 150000
            - logg: e.g. 7.5
            - verbose: if True, some info are printed out
        The wl variable is an array of wavelengths in Angstrom.
        The fl variable is the flux in erg/s/cm2/A
        The variables T and logg are properties: changing them will reload the data
        """
        self.verbose = verbose
        if filename is None:
            if T is not None and logg is not None:
                self.__T = T # We need to initialize the hidden values, as logg is still not defined
                self.logg = logg
                self.filename = '0{0:06.0f}_{1:.2f}_33_50_02_15.bin_0.1.gz'.format(self.T, self.logg)
            else:
                raise TypeError("T and logg must be given")
        else:
            self.filename = filename
            self.__T = float(filename.split('_')[0]) # We need to initialize the hidden values, as logg is still not defined
            self.logg = float(filename.split('_')[1])
        Stel_Spectrum.spec_count += 1
        if self.verbose:
            print('Instantiation done')
            
    def dlfile(self):
        """
        Downloading file if not already here. Put it in the current directory
        """
        if not os.path.exists(self.filename):
            if self.verbose:
                print('Downloading {}'.format(self.filename))
            try:
                stel_file = urlopen('http://astro.uni-tuebingen.de/~rauch/TMAF/NLTE/He+C+N+O/' + 
                                            self.filename)
                output = open(self.filename,'wb')
                output.write(stel_file.read())
                output.close()
                self.file_found=True
            except:
                if self.verbose:
                    print('file {} not found'.format(self.filename))
                self.file_found=False
        else:
            if self.verbose:
                print('{} already on disk'.format(self.filename))
            self.file_found=True
                
    def read_data(self):
        """
        read the data from the file
        """
        if self.file_found:
            data = np.genfromtxt(self.filename, comments='*', names='wl, fl')
            self.fl = data['fl']
            self.wl = data['wl'] # in A
            self.fl /= 1e8 # F LAMBDA  GIVEN IN ERG/CM**2/SEC/CM -> erg/s/cm2/A
            if self.verbose:
                print('Read data from {}'.format(self.filename))
        else:
            if self.verbose:
                print('file not found {}'.format(self.filename))
            self.wl = None
            self.fl = None
        
    def plot_spr(self, ax=None, *args, **kwargs):
        """
        Plot the spectrum.
        Parameter:
            - ax: an axis (optionnal). If Noe or absent, axis is created
            - any extra parameter is passed to ax.plot
        """
        if self.wl is None:
            print('No data to plot')
            return
        if ax is None:
            fig, ax = plt.subplots()
        ax.plot(self.wl, self.fl,
                label='T3={0:.0f}, logg={1}'.format(self.T/1e3, self.logg),
                *args, **kwargs) # Here are the transmissions of extra parameters to plot
        ax.set_yscale('log')
        ax.set_ylim(1e6, 1e14)
        ax.set_xlabel('Wavelength (A)')
        
    def get_integ(self):
        """
        Return the integral of Flambda over lambda, in erg/s/cm2
        """
        if self.wl is None:
            print('No data')
            return None
        return simps(self.fl, self.wl) # perform the integral
    
    def __getT(self): 
        return self.__T
    
    def __setT(self, value): 
        if not isinstance(value, (int, float)): # check the type of the input
            raise TypeError('T must be an integer or a float')
        if float(value) not in np.linspace(40000, 190000, 16): # check the value of the input
            raise ValueError('T value must be between 40000 and 190000K, by 10000K steps')
        elif self.__T != value:
            self.__T = value
            self.filename = '0{0:06.0f}_{1:.2f}_33_50_02_15.bin_0.1.gz'.format(self.T, self.logg)
            self.dlfile() # will download new data
            self.read_data() # will update the data
        
    def __delT(self): 
        print('T is needed')
        
    T = property(__getT, __setT, __delT, "Stellar effective temperature in K")
    
    def __getlogg(self): 
        return self.__logg
    
    def __setlogg(self, value):
        try:
            self.__logg 
        except:
            self.__logg = -1
        if not isinstance(value, (int, float)):
            raise TypeError('logg must be an integer or a float')
        if float(value) not in (-1., 5., 6., 7. ,8., 9.):
            raise ValueError('Error, logg must be 6, 7, 8, or 9')
            self.__logg = None
        elif self.__logg != value:
            self.__logg = value
            self.filename = '0{0:06.0f}_{1:.2f}_33_50_02_15.bin_0.1.gz'.format(self.T, self.logg)
            self.dlfile() # will download new data
            self.read_data() # will update the data
        
    def __dellogg(self): 
        print('logg is needed')
        
    logg = property(__getlogg, __setlogg, __dellogg, "Stellar logg")

    def print_info(self):
        """
        Print out the filename and the number of points
        """
        print(self.__repr__())
        
    def __repr__(self):
        """
        This is what is used when calling "print <obj>" or <obj> ENTER
        """
        if self.wl is None:
            return'Filename: {0}, No data'.format(self.filename)
        else:
            return'Filename: {0}, number of points: {1}'.format(self.filename, len(self.wl))
    
    def __del__(self):
        Stel_Spectrum.spec_count -= 1

spectra = [] # we create an empty list
for T in np.linspace(40000, 190000, 4): # this is the list of available temperature (check the site)
    spectra.append(Stel_Spectrum(T=T, logg=6, verbose=True)) # we fill the list with the objects for each temperature
T = np.array([sp.T for sp in spectra])
F = np.array([sp.get_integ() for sp in spectra])
for t, f in zip(T, F):
    print('Temperature = {0:.0f}K, Flux = {1:.2e} erg/s/cm2'.format(t, f))
print('DONE')

Overwriting test_1_prof.py


In [5]:
%run -t test_1_prof.py

0040000_6.00_33_50_02_15.bin_0.1.gz already on disk
Read data from 0040000_6.00_33_50_02_15.bin_0.1.gz
Instantiation done
0090000_6.00_33_50_02_15.bin_0.1.gz already on disk
Read data from 0090000_6.00_33_50_02_15.bin_0.1.gz
Instantiation done
0140000_6.00_33_50_02_15.bin_0.1.gz already on disk
Read data from 0140000_6.00_33_50_02_15.bin_0.1.gz
Instantiation done
0190000_6.00_33_50_02_15.bin_0.1.gz already on disk
Read data from 0190000_6.00_33_50_02_15.bin_0.1.gz
Instantiation done
Temperature = 40000K, Flux = 4.00e+13 erg/s/cm2
Temperature = 90000K, Flux = 1.05e+15 erg/s/cm2
Temperature = 140000K, Flux = 6.93e+15 erg/s/cm2
Temperature = 190000K, Flux = 2.35e+16 erg/s/cm2
DONE

IPython CPU timings (estimated):
  User   :       0.16 s.
  System :       0.02 s.
Wall time:       0.17 s.


In [7]:
%run -p test_1_prof.py

0040000_6.00_33_50_02_15.bin_0.1.gz already on disk
Read data from 0040000_6.00_33_50_02_15.bin_0.1.gz
Instantiation done
0090000_6.00_33_50_02_15.bin_0.1.gz already on disk
Read data from 0090000_6.00_33_50_02_15.bin_0.1.gz
Instantiation done
0140000_6.00_33_50_02_15.bin_0.1.gz already on disk
Read data from 0140000_6.00_33_50_02_15.bin_0.1.gz
Instantiation done
0190000_6.00_33_50_02_15.bin_0.1.gz already on disk
Read data from 0190000_6.00_33_50_02_15.bin_0.1.gz
Instantiation done
Temperature = 40000K, Flux = 4.00e+13 erg/s/cm2
Temperature = 90000K, Flux = 1.05e+15 erg/s/cm2
Temperature = 140000K, Flux = 6.93e+15 erg/s/cm2
Temperature = 190000K, Flux = 2.35e+16 erg/s/cm2
DONE
 

         964878 function calls (964866 primitive calls) in 0.299 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        4    0.086    0.021    0.317    0.079 npyio.py:1558(genfromtxt)
    79896    0.050    0.000    0.078    0.000 _iotools.py:199(_delimited_splitter)
    79896    0.033    0.000    0.121    0.000 _iotools.py:226(__call__)
   159608    0.024    0.000    0.024    0.000 _iotools.py:671(_loose_call)
       20    0.022    0.001    0.022    0.001 {built-in method numpy.array}
   159721    0.019    0.000    0.019    0.000 {method 'split' of 'str' objects}
   160719    0.011    0.000    0.011    0.000 {built-in method builtins.len}
    79996    0.010    0.000    0.010    0.000 _iotools.py:11(_decode_line)
    79920    0.008    0.000    0.008    0.000 {method 'strip' of 'str' objects}
    80096    0.008    0.000    0.008    0.000 gzip.py:322(closed)
    79853    0.006    0.000    0.006    0.000 {method 'append' of 'l

## Have a look at Scalene https://github.com/emeryberger/scalene

In [6]:
%scrun spectra = Stel_Spectrum(T=100000, logg=6, verbose=True)

0100000_6.00_33_50_02_15.bin_0.1.gz already on disk
Read data from 0100000_6.00_33_50_02_15.bin_0.1.gz
Instantiation done


In [9]:
%%scalene 

spectra = Stel_Spectrum(T=100000, logg=6, verbose=True)
I =  spectra.get_integ()
print(I)
print('DONE')

Read data from 0100000_6.00_33_50_02_15.bin_0.1.gz
Instantiation done
1790757180815459.2
DONE


In [18]:
class Stel_Spectrum(object):
    """
    This object downloads a file from http://astro.uni-tuebingen.de/~rauch/TMAF/NLTE/He+C+N+O/
    and is able to make some plots.    
    """
    
    spec_count = 0 # This attibute is at the level of the class, not of the object.
    def __init__(self, filename=None, T=None, logg=None, verbose=False):
        """
        Initialisation of the Stel_Spectrum object.
        Parameter:
            - filename
            - T: temperature in K, e.g. 150000
            - logg: e.g. 7.5
            - verbose: if True, some info are printed out
        The wl variable is an array of wavelengths in Angstrom.
        The fl variable is the flux in erg/s/cm2/A
        The variables T and logg are properties: changing them will reload the data
        """
        self.verbose = verbose
        if filename is None:
            if T is not None and logg is not None:
                self.__T = T # We need to initialize the hidden values, as logg is still not defined
                self.logg = logg
                self.filename = '0{0:06.0f}_{1:.2f}_33_50_02_15.bin_0.1.gz'.format(self.T, self.logg)
            else:
                raise TypeError("T and logg must be given")
        else:
            self.filename = filename
            self.__T = float(filename.split('_')[0]) # We need to initialize the hidden values, as logg is still not defined
            self.logg = float(filename.split('_')[1])
        Stel_Spectrum.spec_count += 1
        if self.verbose:
            print('Instantiation done')
            
    def dlfile(self):
        """
        Downloading file if not already here. Put it in the current directory
        """
        if not os.path.exists(self.filename):
            if self.verbose:
                print('Downloading {}'.format(self.filename))
            try:
                stel_file = urlopen('http://astro.uni-tuebingen.de/~rauch/TMAF/NLTE/He+C+N+O/' + 
                                            self.filename)
                output = open(self.filename,'wb')
                output.write(stel_file.read())
                output.close()
                self.file_found=True
            except:
                if self.verbose:
                    print('file {} not found'.format(self.filename))
                self.file_found=False
        else:
            if self.verbose:
                print('{} already on disk'.format(self.filename))
            self.file_found=True
    def read_data(self):
        """
        read the data from the file
        """
        if self.file_found:
            data = np.genfromtxt(self.filename, comments='*', names='wl, fl')
            self.fl = data['fl']
            self.wl = data['wl'] # in A
            self.fl /= 1e8 # F LAMBDA  GIVEN IN ERG/CM**2/SEC/CM -> erg/s/cm2/A
            if self.verbose:
                print('Read data from {}'.format(self.filename))
        else:
            if self.verbose:
                print('file not found {}'.format(self.filename))
            self.wl = None
            self.fl = None
        
    def plot_spr(self, ax=None, *args, **kwargs):
        """
        Plot the spectrum.
        Parameter:
            - ax: an axis (optionnal). If Noe or absent, axis is created
            - any extra parameter is passed to ax.plot
        """
        if self.wl is None:
            print('No data to plot')
            return
        if ax is None:
            fig, ax = plt.subplots()
        ax.plot(self.wl, self.fl,
                label='T3={0:.0f}, logg={1}'.format(self.T/1e3, self.logg),
                *args, **kwargs) # Here are the transmissions of extra parameters to plot
        ax.set_yscale('log')
        ax.set_ylim(1e6, 1e14)
        ax.set_xlabel('Wavelength (A)')
        
    def get_integ(self):
        """
        Return the integral of Flambda over lambda, in erg/s/cm2
        """
        if self.wl is None:
            print('No data')
            return None
        return simps(self.fl, self.wl) # perform the integral
    
    def __getT(self): 
        return self.__T
    
    def __setT(self, value): 
        if not isinstance(value, (int, float)): # check the type of the input
            raise TypeError('T must be an integer or a float')
        if float(value) not in np.linspace(40000, 190000, 16): # check the value of the input
            raise ValueError('T value must be between 40000 and 190000K, by 10000K steps')
        elif self.__T != value:
            self.__T = value
            self.filename = '0{0:06.0f}_{1:.2f}_33_50_02_15.bin_0.1.gz'.format(self.T, self.logg)
            self.dlfile() # will download new data
            self.read_data() # will update the data
        
    def __delT(self): 
        print('T is needed')
        
    T = property(__getT, __setT, __delT, "Stellar effective temperature in K")
    
    def __getlogg(self): 
        return self.__logg
    
    def __setlogg(self, value):
        try:
            self.__logg 
        except:
            self.__logg = -1
        if not isinstance(value, (int, float)):
            raise TypeError('logg must be an integer or a float')
        if float(value) not in (-1., 5., 6., 7. ,8., 9.):
            raise ValueError('Error, logg must be 6, 7, 8, or 9')
            self.__logg = None
        elif self.__logg != value:
            self.__logg = value
            self.filename = '0{0:06.0f}_{1:.2f}_33_50_02_15.bin_0.1.gz'.format(self.T, self.logg)
            self.dlfile() # will download new data
            self.read_data() # will update the data
        
    def __dellogg(self): 
        print('logg is needed')
        
    logg = property(__getlogg, __setlogg, __dellogg, "Stellar logg")

    def print_info(self):
        """
        Print out the filename and the number of points
        """
        print(self.__repr__())
        
    def __repr__(self):
        """
        This is what is used when calling "print <obj>" or <obj> ENTER
        """
        if self.wl is None:
            return'Filename: {0}, No data'.format(self.filename)
        else:
            return'Filename: {0}, number of points: {1}'.format(self.filename, len(self.wl))
    
    def __del__(self):
        Stel_Spectrum.spec_count -= 1

In [26]:
spectra = Stel_Spectrum(T=100000, logg=6, verbose=True)

0100000_6.00_33_50_02_15.bin_0.1.gz already on disk
Read data from 0100000_6.00_33_50_02_15.bin_0.1.gz
Instantiation done


In [22]:
%%scalene 

spectra = Stel_Spectrum(T=100000, logg=6, verbose=True)
I =  spectra.get_integ()
print(I)
print('DONE')

0100000_6.00_33_50_02_15.bin_0.1.gz already on disk
Read data from 0100000_6.00_33_50_02_15.bin_0.1.gz
Instantiation done
1790757180815459.2
DONE


## Debugger

In [1]:
import numpy as np
import os
from urllib.request import urlopen
from scipy.integrate import simps

#### From the terminal

#### Breakpoint

In [2]:
# import pdb # need to call the debugger at the breakpoint
# Inserting a pdb.set_trace() in the __init__ method to stop the program and inspect it

In [14]:
%%writefile test_5_pdb.py
import pdb # This is needed to use the debugger
import numpy as np
import os
from urllib.request import urlopen
from scipy.integrate import simps

class Stel_Spectrum(object):
    """
    This object downloads a file from http://astro.uni-tuebingen.de/~rauch/TMAF/NLTE/He+C+N+O/
    and is able to make some plots.    
    """
    
    spec_count = 0 # This attibute is at the level of the class, not of the object.
    def __init__(self, filename=None, T=None, logg=None, verbose=False):
        """
        Initialisation of the Stel_Spectrum object.
        Parameter:
            - filename
            - T: temperature in K, e.g. 150000
            - logg: e.g. 7.5
            - verbose: if True, some info are printed out
        The wl variable is an array of wavelengths in Angstrom.
        The fl variable is the flux in erg/s/cm2/A
        The variables T and logg are properties: changing them will reload the data
        """
        self.verbose = verbose
        if filename is None:
            if T is not None and logg is not None:
                self.__T = T # We need to initialize the hidden values, as logg is still not defined
                self.logg = logg
                self.filename = '0{0:06.0f}_{1:.2f}_33_50_02_15.bin_0.1.gz'.format(self.T, self.logg)
            else:
                raise TypeError("T and logg must be given")
        else:
            self.filename = filename
            self.__T = float(filename.split('_')[0]) # We need to initialize the hidden values, as logg is still not defined
            self.logg = float(filename.split('_')[1])
        Stel_Spectrum.spec_count += 1
        if self.verbose:
            print('Instantiation done')
            
    def dlfile(self):
        """
        Downloading file if not already here. Put it in the current directory
        """
        if not os.path.exists(self.filename):
            if self.verbose:
                print('Downloading {}'.format(self.filename))
            try:
                stel_file = urlopen('http://astro.uni-tuebingen.de/~rauch/TMAF/NLTE/He+C+N+O/' + 
                                            self.filename)
                output = open(self.filename,'wb')
                output.write(stel_file.read())
                output.close()
                self.file_found=True
            except:
                if self.verbose:
                    print('file {} not found'.format(self.filename))
                self.file_found=False
        else:
            if self.verbose:
                print('{} already on disk'.format(self.filename))
            self.file_found=True
                
    def read_data(self):
        """
        read the data from the file
        """
        if self.file_found:
            file2read = self.filename+'eee'
            pdb.set_trace() # THIS IS A BREAKPOINT
            data = np.genfromtxt(file2read, comments='*', names='wl, fl')
            self.fl = data['fl']
            self.wl = data['wl'] # in A
            self.fl /= 1e8 # F LAMBDA  GIVEN IN ERG/CM**2/SEC/CM -> erg/s/cm2/A
            if self.verbose:
                print('Read data from {}'.format(self.filename))
        else:
            if self.verbose:
                print('file not found {}'.format(self.filename))
            self.wl = None
            self.fl = None
        
    def plot_spr(self, ax=None, *args, **kwargs):
        """
        Plot the spectrum.
        Parameter:
            - ax: an axis (optionnal). If Noe or absent, axis is created
            - any extra parameter is passed to ax.plot
        """
        if self.wl is None:
            print('No data to plot')
            return
        if ax is None:
            fig, ax = plt.subplots()
        ax.plot(self.wl, self.fl,
                label='T3={0:.0f}, logg={1}'.format(self.T/1e3, self.logg),
                *args, **kwargs) # Here are the transmissions of extra parameters to plot
        ax.set_yscale('log')
        ax.set_ylim(1e6, 1e14)
        ax.set_xlabel('Wavelength (A)')
        
    def get_integ(self):
        """
        Return the integral of Flambda over lambda, in erg/s/cm2
        """
        if self.wl is None:
            print('No data')
            return None
        return simps(self.fl, self.wl) # perform the integral
    
    def __getT(self): 
        return self.__T
    
    def __setT(self, value): 
        if not isinstance(value, (int, float)): # check the type of the input
            raise TypeError('T must be an integer or a float')
        if float(value) not in np.linspace(40000, 190000, 16): # check the value of the input
            raise ValueError('T value must be between 40000 and 190000K, by 10000K steps')
        elif self.__T != value:
            self.__T = value
            self.filename = '0{0:06.0f}_{1:.2f}_33_50_02_15.bin_0.1.gz'.format(self.T, self.logg)
            self.dlfile() # will download new data
            self.read_data() # will update the data
        
    def __delT(self): 
        print('T is needed')
        
    T = property(__getT, __setT, __delT, "Stellar effective temperature")
    
    def __getlogg(self): 
        return self.__logg
    
    def __setlogg(self, value):
        try:
            self.__logg 
        except:
            self.__logg = -1
        if not isinstance(value, (int, float)):
            raise TypeError('logg must be an integer or a float')
        if float(value) not in (-1., 5., 6., 7. ,8., 9.):
            raise ValueError('Error, logg must be 6, 7, 8, or 9')
            self.__logg = None
        elif self.__logg != value:
            self.__logg = value
            self.filename = '0{0:06.0f}_{1:.2f}_33_50_02_15.bin_0.1.gz'.format(self.T, self.logg)
            self.dlfile() # will download new data
            self.read_data() # will update the data
        
    def __dellogg(self): 
        print('logg is needed')
        
    logg = property(__getlogg, __setlogg, __dellogg, "Stellar logg")

    def print_info(self):
        """
        Print out the filename and the number of points
        """
        print(self.__repr__())
        
    def __repr__(self):
        """
        This is what is used when calling "print <obj>" or <obj> ENTER
        """
        if self.wl is None:
            return'Filename: {0}, No data'.format(self.filename)
        else:
            return'Filename: {0}, number of points: {1}'.format(self.filename, len(self.wl))
    
    def __del__(self):
        Stel_Spectrum.spec_count -= 1

sp = Stel_Spectrum(T=100000, logg=6)
print('ending')
print(sp.filename, sp.get_integ())

Overwriting test_5_pdb.py


The commands that can be used once inside the pdb debugger session are: 
* l(list)	Lists the code at the current position
* u(p)	    Walk up the call stack
* d(own)	Walk down the call stack
* n(ext)	Execute the next line (does not go down in new functions)
* s(tep)	Execute the next statement (goes down in new functions)
* bt	    Print the call stack
* a	        Print the local variables
* !command	Execute the given Python command (by opposition to pdb commands
* break N   Set a breakpoint at line number N. If no N, list all the breakpoints
* disable N Remove the breakpoin number N
* c(ontinue) Run until the next breakpoint or the end of the program
* return   Continues executing until the function is about to execute a return statement, and then it pauses. This gives you time to look at the return value before the function returns.

In [12]:
%run test_5_pdb.py

> /home/morisset/Python-lectures-Notebooks/Notebooks/test_5_pdb.py(72)read_data()
     70             file2read = self.filename+'eee'
     71             pdb.set_trace() # THIS IS A BREAKPOINT
---> 72             data = np.genfromtxt(file2read, comments='*', names='wl, fl')
     73             self.fl = data['fl']
     74             self.wl = data['wl'] # in A

--KeyboardInterrupt--

KeyboardInterrupt: Interrupted by user


OSError: 0100000_6.00_33_50_02_15.bin_0.1.gzeee not found.

In [18]:
%%writefile test_2_err.py
import numpy as np
import os
from urllib.request import urlopen
from scipy.integrate import simps

class Stel_Spectrum(object):
    """
    This object downloads a file from http://astro.uni-tuebingen.de/~rauch/TMAF/NLTE/He+C+N+O/
    and is able to make some plots.    
    """
    
    spec_count = 0 # This attibute is at the level of the class, not of the object.
    def __init__(self, filename=None, T=None, logg=None, verbose=False):
        """
        Initialisation of the Stel_Spectrum object.
        Parameter:
            - filename
            - T: temperature in K, e.g. 150000
            - logg: e.g. 7.5
            - verbose: if True, some info are printed out
        The wl variable is an array of wavelengths in Angstrom.
        The fl variable is the flux in erg/s/cm2/A
        The variables T and logg are properties: changing them will reload the data
        """
        self.verbose = verbose
        if filename is None:
            if T is not None and logg is not None:
                self.__T = T # We need to initialize the hidden values, as logg is still not defined
                self.logg = logg
                self.filename = '0{0:06.0f}_{1:.2f}_33_50_02_15.bin_0.1.gz'.format(self.T, self.logg)
            else:
                raise TypeError("T and logg must be given")
        else:
            self.filename = filename
            self.__T = float(filename.split('_')[0]) # We need to initialize the hidden values, as logg is still not defined
            self.logg = float(filename.split('_')[1])
        Stel_Spectrum.spec_count += 1
        if self.verbose:
            print('Instantiation done')
            
    def dlfile(self):
        """
        Downloading file if not already here. Put it in the current directory
        """
        if not os.path.exists(self.filename):
            if self.verbose:
                print('Downloading {}'.format(self.filename))
            try:
                stel_file = urlopen('http://astro.uni-tuebingen.de/~rauch/TMAF/NLTE/He+C+N+O/' + 
                                            self.filename)
                output = open(self.filename,'wb')
                output.write(stel_file.read())
                output.close()
                self.file_found=True
            except:
                if self.verbose:
                    print('file {} not found'.format(self.filename))
                self.file_found=False
        else:
            if self.verbose:
                print('{} already on disk'.format(self.filename))
            self.file_found=True
                
    def read_data(self):
        """
        read the data from the file
        """
        if self.file_found:
            file2read = self.filename+'eee'
            data = np.genfromtxt(file2read, comments='*', names='wl, fl')
            self.fl = data['fl']
            self.wl = data['wl'] # in A
            self.fl /= 1e8 # F LAMBDA  GIVEN IN ERG/CM**2/SEC/CM -> erg/s/cm2/A
            if self.verbose:
                print('Read data from {}'.format(self.filename))
        else:
            if self.verbose:
                print('file not found {}'.format(self.filename))
            self.wl = None
            self.fl = None
        
    def plot_spr(self, ax=None, *args, **kwargs):
        """
        Plot the spectrum.
        Parameter:
            - ax: an axis (optionnal). If Noe or absent, axis is created
            - any extra parameter is passed to ax.plot
        """
        if self.wl is None:
            print('No data to plot')
            return
        if ax is None:
            fig, ax = plt.subplots()
        ax.plot(self.wl, self.fl,
                label='T3={0:.0f}, logg={1}'.format(self.T/1e3, self.logg),
                *args, **kwargs) # Here are the transmissions of extra parameters to plot
        ax.set_yscale('log')
        ax.set_ylim(1e6, 1e14)
        ax.set_xlabel('Wavelength (A)')
        
    def get_integ(self):
        """
        Return the integral of Flambda over lambda, in erg/s/cm2
        """
        if self.wl is None:
            print('No data')
            return None
        return simps(self.fl, self.wl) # perform the integral
    
    def __getT(self): 
        return self.__T
    
    def __setT(self, value): 
        if not isinstance(value, (int, float)): # check the type of the input
            raise TypeError('T must be an integer or a float')
        if float(value) not in np.linspace(40000, 190000, 16): # check the value of the input
            raise ValueError('T value must be between 40000 and 190000K, by 10000K steps')
        elif self.__T != value:
            self.__T = value
            self.filename = '0{0:06.0f}_{1:.2f}_33_50_02_15.bin_0.1.gz'.format(self.T, self.logg)
            self.dlfile() # will download new data
            self.read_data() # will update the data
        
    def __delT(self): 
        print('T is needed')
        
    T = property(__getT, __setT, __delT, "Stellar effective temperature")
    
    def __getlogg(self): 
        return self.__logg
    
    def __setlogg(self, value):
        try:
            self.__logg 
        except:
            self.__logg = -1
        if not isinstance(value, (int, float)):
            raise TypeError('logg must be an integer or a float')
        if float(value) not in (-1., 5., 6., 7. ,8., 9.):
            raise ValueError('Error, logg must be 6, 7, 8, or 9')
            self.__logg = None
        elif self.__logg != value:
            self.__logg = value
            self.filename = '0{0:06.0f}_{1:.2f}_33_50_02_15.bin_0.1.gz'.format(self.T, self.logg)
            self.dlfile() # will download new data
            self.read_data() # will update the data
        
    def __dellogg(self): 
        print('logg is needed')
        
    logg = property(__getlogg, __setlogg, __dellogg, "Stellar logg")

    def print_info(self):
        """
        Print out the filename and the number of points
        """
        print(self.__repr__())
        
    def __repr__(self):
        """
        This is what is used when calling "print <obj>" or <obj> ENTER
        """
        if self.wl is None:
            return'Filename: {0}, No data'.format(self.filename)
        else:
            return'Filename: {0}, number of points: {1}'.format(self.filename, len(self.wl))
    
    def __del__(self):
        Stel_Spectrum.spec_count -= 1

sp = Stel_Spectrum(T=100000, logg=6, verbose=True)
print('ending')
print(sp.filename, sp.get_integ())

Overwriting test_2_err.py


In [ ]:
# ! ipython -m pdb test_2_err.py # from a terminal